In [1]:
from boink.hashing import UKHShifter
from boink.hashing import RollingHashShifter
from debruijnal_enhance_o_tron.sequence import kmers
from debruijnal_enhance_o_tron.sequence import get_random_sequence

In [5]:
class dBG:
    '''A basic de Bruijn Graph.
    '''

    def __init__(self, ksize, Sigma='ACGT', *args, **kwargs):
        self.store = set()
        self.tags = set()
        self.ksize = ksize
        self.Sigma = Sigma
        self.Sigma_set = set(self.Sigma)
        self.Sigma_list = list(self.Sigma)
        
    def kmers(self, sequence):
        '''Yields the k-mers in the given sequence.
        '''
        for i in range(len(sequence) - self.ksize + 1):
            yield sequence[i:i+self.ksize]

    def reset(self):
        '''Reset the dBG to an empty state.
        '''
        self.store.clear()

    def get(self, item):
        '''Return true if the item is in the dBG, false otherwise
        '''
        return item in self.store

    def add(self, item):
        '''Add the k-mer(s) from the given sequence to the dBG.
        '''
        if len(item) < self.ksize:
            raise ValueError(item)
        elif len(item) == self.ksize:
            self.store.add(item)
        else:
            self.store.update(self.kmers(item))
    
    def tag(self, item, dist=3):
        if len(item) < self.ksize:
            raise ValueError(item)
        count = 0
        n_tagged = 0
        for kmer in self.kmers(item):
            if kmer in self.tags:
                count = 0
            else:
                count += 1
                if count > dist:
                    self.tags.add(kmer)
                    count = 0
                    n_tagged += 1
        return n_tagged
    
    def remove(self, item):
        '''Remove the k-mer(s) from the given sequence from the dBG.
        '''
        if len(item) < self.ksize:
            raise ValueError(item)
        elif len(item) == self.ksize:
            self.store.remove(item)
        else:
            self.store.difference_update(self.kmers(item))
    
    def suffix(self, sequence):
        '''Returns the length k-1 suffix of the sequence.
        '''
        if len(sequence) < self.ksize:
            raise ValueError('Sequence too short')
        return sequence[-self.ksize+1:]
    
    def prefix(self, sequence):
        '''Returns the length k-1 prefix of the sequence.
        '''
        if len(sequence) < self.ksize:
            raise ValueError('Sequence too short')
        return sequence[:self.ksize-1]
    
    def left_neighbors(self, item):
        '''Yields the four left neighbors of the given k-mer.
        '''
        prefix = self.prefix(item)
        for b in self.Sigma:
            yield b + prefix
    
    def right_neighbors(self, item):
        '''Yields the four right neighbors of the given k-mer.
        '''
        suffix = self.suffix(item)
        for b in self.Sigma:
            yield suffix + b

    def left_degree(self, item):
        '''Yields the in-degree (left degree) of the given k-mer.
        '''
        return sum((self.get(neighbor) for neighbor in self.left_neighbors(item)))

    def right_degree(self, item):
        '''Yields the out-degree (right degree) of the given k-mer.
        '''
        return sum((self.get(neighbor) for neighbor in self.right_neighbors(item)))
    
    def degree(self, item):
        return self.left_degree(item), self.right_degree(item)
    
    def is_decision(self, kmer):
        '''Returns True if the given k-mer is a decision k-mer in the dBG:
        that is, if its in-degree or out-degree is greater than one.
        '''
        return self.left_degree(kmer) > 1 or self.right_degree(kmer) > 1

    def find_decisions(self, sequence):
        '''Yields the position and k-mer sequence of the decision k-mers in the 
        given sequence.
        '''
        for n, kmer in enumerate(self.kmers(sequence)):
            if self.is_decision(kmer):
                yield n, kmer

In [3]:
import bbhash

In [2]:
ukhs = UKHShifter(27, 7)

UKHS: using W=20, K=7


In [3]:
seq = get_random_sequence(100, 27)

In [4]:
ukhs.find_unikmers(seq)

b'ACAGTACAGTCAAGCGATACTCCGAGCTTAACCGGACAACGGTTCTCGCCTATCGCCCTTCACCTCAATTGGG'


[(4287246436847025391, 850),
 (4287246436847025391, 850),
 (4287246436847025391, 850),
 (4287246436847025391, 850),
 (6756425901368628185, 990),
 (6756425901368628185, 990),
 (6756425901368628185, 990),
 (6756425901368628185, 990),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (26644528043774785, 3264),
 (3374902433127799277, 1075),
 (3374902433127799277, 1075),
 (923109349927235793, 1331),
 (923109349927235793, 1331),
 (923109349927235793, 1331),
 (923109349927235793, 1331),
 (92

In [6]:
ukhs.set_cursor(seq)

In [7]:
seq

'CGTGCAGTTCTAAATTCGAAGCCTGCAGGCTTGGTCGTGGGTTAAAGAATTTACTGTACACCAGCACATACGGCACCAGCTAATAAAAATACTTAAACAA'

In [15]:
for kmer in kmers(seq, 7):
    print('(', hasher.hash(kmer), ', ', ukhs.query(hasher.hash(kmer)), ')', sep='')

(10973492759661448830, None)
(9508044623959221480, None)
(5551789466241464427, None)
(9981562942641867623, None)
(6910235179741095797, None)
(16173567458420033444, 773)
(9681429984473271322, None)
(12286883827231201492, None)
(10628437211489627735, None)
(16179131663506539345, None)
(11855042890701003513, None)
(462264805413977041, None)
(5361487379720942833, 1435)
(15167255132354652849, None)
(9767070260505360055, None)
(2271294049989203678, None)
(3429763133724022285, None)
(17028648904892568591, None)
(1970231056644343929, None)
(5345211148680906421, None)
(15199438622384311865, 21)
(7361969857450458133, None)
(17738795196983906787, None)
(9931439664741875613, None)
(15411137244630950085, None)
(16236571949337440305, None)
(17933022849944390058, None)
(15190505698020061313, None)
(17253953829566599353, None)
(4819930053698499718, 957)
(13816626555898921157, 155)
(13332147360646422133, None)
(4425900195240702665, None)
(13340665749868037620, None)
(12804676011767737231, None)
(118982

In [8]:
ukhs.find_unikmers(seq)

b'GGCTTGGTCGTGGGTTAAAGAATTTACTGTACACCAGCACATACGGCACCAGCTAATAAAAATACTTAAACAA'


[(5361487379720942833, 1435),
 (5361487379720942833, 1435),
 (5361487379720942833, 1435),
 (5361487379720942833, 1435),
 (5361487379720942833, 1435),
 (5361487379720942833, 1435),
 (5361487379720942833, 1435),
 (5361487379720942833, 1435),
 (5361487379720942833, 1435),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (4819930053698499718, 957),
 (6092996541620212556, 2619),
 (6092996541620212556, 2619),
 (6092996541620212556, 2619),
 (6092996541620212556, 2619),
 

In [6]:
c_ukhs = ukhs.hashes

In [7]:
mph = bbhash.PyMPHF(c_ukhs, len(c_ukhs), 1, 2.0)

In [12]:
for kmer in kmers(seq, 7):
    h = hasher.hash(kmer)
    print(h, kmer, ukhs.query(h), h in c_ukhs, mph.lookup(h))

3966585810826196083 TCACCTG None False 1195
12132712385095546648 CACCTGT None False 2647
7690237574936954872 ACCTGTG None False 163
9158360649715908088 CCTGTGT None False 2034
5571541805904823269 CTGTGTT 1681 True 1681
11591423859377237507 TGTGTTG None False 2715
9782452822128919545 GTGTTGT 1219 True 1219
14731550741190813589 TGTTGTG 1250 True 1250
9832345464020448410 GTTGTGC None False 2180
6488094177847892111 TTGTGCT 1980 True 1980
11435990624582328251 TGTGCTA None False 2830
3313502993816634566 GTGCTAC None False 2009
1232448785995184182 TGCTACT None False 462
16450901114743926674 GCTACTT None False None
823832800960675651 CTACTTG None False 410
8311279440186057436 TACTTGC None False 2884
12235821944465034138 ACTTGCG 1137 True 1137
8137706075476292833 CTTGCGG None False 43
9536326726933558680 TTGCGGC None False 1179
5174287711837398291 TGCGGCG None False None
11461645855627522455 GCGGCGC None False None


In [13]:
ukhs.set_cursor(seq)

In [14]:
ukhs.unikmer

5571541805904823269

In [15]:
ukhs.partition

1681

In [16]:
seq

'TCACCTGTGTTGTGCTACTTGCGGCGC'

In [17]:
ukhs.hashvalue

13194817695400542713

In [45]:
hpp = ''
cpp = ''
for W in range(20, 210, 10):
    for K in range(7, 10):
        cpp += '    const std::vector<std::string> UKHSData::W{0}_K{1} = {{'.format(W, K)
        hpp += '    static const std::vector<std::string> W{0}_K{1};\n'.format(W, K)
        with open('../boink/ukhs-data/res_{0}_{1}_4_0.txt'.format(K, W)) as fp:          
            for n, line in enumerate(fp):
                kmer = line.strip()
                if n != 0:
                    cpp += ', '
                cpp += '"{0}"'.format(kmer)
        cpp += '};\n'

In [40]:
print(hpp)

    static const std::vector<std::string> W20_K7;
    static const std::vector<std::string> W20_K8;
    static const std::vector<std::string> W20_K9;
    static const std::vector<std::string> W30_K7;
    static const std::vector<std::string> W30_K8;
    static const std::vector<std::string> W30_K9;
    static const std::vector<std::string> W40_K7;
    static const std::vector<std::string> W40_K8;
    static const std::vector<std::string> W40_K9;
    static const std::vector<std::string> W50_K7;
    static const std::vector<std::string> W50_K8;
    static const std::vector<std::string> W50_K9;
    static const std::vector<std::string> W60_K7;
    static const std::vector<std::string> W60_K8;
    static const std::vector<std::string> W60_K9;
    static const std::vector<std::string> W70_K7;
    static const std::vector<std::string> W70_K8;
    static const std::vector<std::string> W70_K9;
    static const std::vector<std::string> W80_K7;
    static const std::vector<std::string> W80_K8;


In [36]:
func = 'const std::vector<std::string>& get_ukhs(int W, int K) {\n    if '
for W in range(20, 210, 10):
    for K in range(7, 10):
        func += '(W == {W} && K == {K}) {{\n        return W{W}_K{K};\n    }} else if '.format(W=W, K=K)
func += '\n}'

In [37]:
print(func)

const std::vector<std::string>& get_ukhs(int W, int K) {
    if (W == 20 && K == 7) {
        return W20_K7;
    } else if (W == 20 && K == 8) {
        return W20_K8;
    } else if (W == 20 && K == 9) {
        return W20_K9;
    } else if (W == 30 && K == 7) {
        return W30_K7;
    } else if (W == 30 && K == 8) {
        return W30_K8;
    } else if (W == 30 && K == 9) {
        return W30_K9;
    } else if (W == 40 && K == 7) {
        return W40_K7;
    } else if (W == 40 && K == 8) {
        return W40_K8;
    } else if (W == 40 && K == 9) {
        return W40_K9;
    } else if (W == 50 && K == 7) {
        return W50_K7;
    } else if (W == 50 && K == 8) {
        return W50_K8;
    } else if (W == 50 && K == 9) {
        return W50_K9;
    } else if (W == 60 && K == 7) {
        return W60_K7;
    } else if (W == 60 && K == 8) {
        return W60_K8;
    } else if (W == 60 && K == 9) {
        return W60_K9;
    } else if (W == 70 && K == 7) {
        return W70_K7;
    } 

In [46]:
with open('defs.cc', 'w') as fp:
    print(cpp, file=fp)